# Lesson 5: Email Assistant with Semantic + Episodic + Procedural Memory

We previously built an email assistant that:
- Classifies incoming messages (respond, ignore, notify)
- Uses human-in-the-loop to refine the assistant's ability to classify emails
- Drafts responses
- Schedules meetings
- Uses memory to remember details from previous emails

Now, we'll add procedural memory that allows the user to update instructions for using the calendar and email writing tools.

In [2]:
from langgraph.store.memory import InMemoryStore
from models import get_embeddings_model

store = InMemoryStore(
    index={"embed": get_embeddings_model()}
)

config = {"configurable": {"langgraph_user_id": "lance"}}
langgraph_user_id = config["configurable"]["langgraph_user_id"]

In [3]:
from models import get_foundation_model, get_router_model

llm = get_foundation_model()
llm_router = get_router_model(llm)

In [4]:
from langgraph.graph import add_messages
from typing_extensions import TypedDict, Annotated

class State(TypedDict):
    email_input: dict
    messages: Annotated[list, add_messages]

## Updated triage_router gets ignore, notify and respond rule from store

In [5]:
from helper import format_few_shot_examples

def get_email_examples(namespace: dict, state: State) -> str:
    examples = store.search(
        namespace, 
        query=str({"email": state['email_input']})
    ) 
    return format_few_shot_examples(examples)

In [6]:
def get_ignore_prompt(namespace: dict, prompt_instructions: str, store: InMemoryStore) -> str:
    result = store.get(namespace, "triage_ignore")
    if result is None:
        store.put(
            namespace, 
            "triage_ignore", 
            {"prompt": prompt_instructions["triage_rules"]["ignore"]}
        )
        return prompt_instructions["triage_rules"]["ignore"]
    else:
        return result.value['prompt']

In [7]:
def get_respond_prompt(namespace: dict, prompt_instructions: str, store: InMemoryStore) -> str:
    result = store.get(namespace, "triage_respond")
    if result is None:
        store.put(
            namespace, 
            "triage_respond", 
            {"prompt": prompt_instructions["triage_rules"]["respond"]}
        )
        return prompt_instructions["triage_rules"]["respond"]
    else:
        return result.value['prompt']

In [8]:
def get_notify_prompt(namespace: dict, prompt_instructions: str, store: InMemoryStore) -> str:
    result = store.get(namespace, "triage_notify")
    if result is None:
        store.put(
            namespace, 
            "triage_notify", 
            {"prompt": prompt_instructions["triage_rules"]["notify"]}
        )
        return prompt_instructions["triage_rules"]["notify"]
    else:
        return result.value['prompt']

In [9]:
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from typing import Literal
from IPython.display import Image, display

from prompts import triage_user_prompt
from profiles import john_doe_profile as profile
from prompts_instructions import prompt_instructions
from prompts import triage_system_prompt

def triage_router(state: State, config, store) -> Command[
    Literal["response_agent", "__end__"]
]:
    namespace = (
        "email_assistant",
        langgraph_user_id,
        "examples"
    )
    examples = get_email_examples(config, state)

    ## Changes - START ##
    namespace = (langgraph_user_id, )

    ignore_prompt = get_ignore_prompt(namespace, prompt_instructions, store)
    notify_prompt = get_notify_prompt(namespace, prompt_instructions, store)
    respond_prompt = get_respond_prompt(namespace, prompt_instructions, store)
    ## Changes - END ##

    system_prompt = triage_system_prompt.format(
        full_name=profile["full_name"],
        name=profile["name"],
        user_profile_background=profile["user_profile_background"],
        ## Changes - START ##
        triage_no=ignore_prompt,
        triage_notify=notify_prompt,
        triage_email=respond_prompt,
        ## Changes - END ##
        examples=examples
    )
    user_prompt = triage_user_prompt.format(
        author=state['email_input']['author'], 
        to=state['email_input']['to'], 
        subject=state['email_input']['subject'], 
        email_thread=state['email_input']['email_thread']
    )

    result = llm_router.invoke(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    )

    if result.classification == "respond":
        print("📧 Classification: RESPOND - This email requires a response")
        goto = "response_agent"
        update = {
            "messages": [
                {
                    "role": "user",
                    "content": f"Respond to the email {state['email_input']}",
                }
            ]
        }
    elif result.classification == "ignore":
        print("🚫 Classification: IGNORE - This email can be safely ignored")
        update = None
        goto = END
    elif result.classification == "notify":
        # If real life, this would do something else
        print("🔔 Classification: NOTIFY - This email contains important information")
        update = None
        goto = END
    else:
        raise ValueError(f"Invalid classification: {result.classification}")

    return Command(goto=goto, update=update)

## Updated get_create_prompt_func gets prompt from store

In [10]:
def get_instructions_prompt(
    namespace: dict, 
    prompt_instructions: str, 
    store: InMemoryStore
) -> str:
    result = store.get(namespace, "agent_instructions")
    if result is None:
        store.put(
            namespace, 
            "agent_instructions", 
            {"prompt": prompt_instructions["agent_instructions"]}
        )
        return prompt_instructions["agent_instructions"]
    else:
        return result.value['prompt']

In [11]:
def get_create_prompt_func(system_prompt, prompt_instructions, profile, langgraph_user_id, store):
    namespace = (langgraph_user_id, )
    prompt = get_instructions_prompt(namespace, prompt_instructions, store)

    return lambda state: [
        {
            "role": "system", 
            "content": system_prompt.format(
                instructions=prompt, 
                **profile
            )
        }
    ] + state['messages']
    

# Create the email agent

In [12]:
agent_system_prompt_memory = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Tools >
You have access to the following tools to help manage {name}'s communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day) - Schedule calendar meetings
3. check_calendar_availability(day) - Check available time slots for a given day
4. manage_memory - Store any relevant information about contacts, actions, discussion, etc. in memory for future reference
5. search_memory - Search for any relevant information that may have been stored in memory
</ Tools >

< Instructions >
{instructions}
</ Instructions >
"""

In [13]:
from langmem import create_manage_memory_tool, create_search_memory_tool

manage_memory_tool = create_manage_memory_tool(
    namespace=(
        "email_assistant", 
        "{langgraph_user_id}",
        "collection"
    )
)
search_memory_tool = create_search_memory_tool(
    namespace=(
        "email_assistant",
        "{langgraph_user_id}",
        "collection"
    )
)

In [14]:
from langgraph.prebuilt import create_react_agent
from tools import write_email, schedule_meeting, check_calendar_availability
from profiles import john_doe_profile as profile

tools= [
    write_email, 
    schedule_meeting,
    check_calendar_availability,
    manage_memory_tool,
    search_memory_tool
]
response_agent = create_react_agent(
    # Gemini foundation model
    llm,
    tools=tools,
    ## Changes - START ##
    prompt=get_create_prompt_func(agent_system_prompt_memory, prompt_instructions, profile, langgraph_user_id, store),
    ## Changes - END ##
    # Use this to ensure the store is passed to the agent 
    store=store
)

In [15]:
email_agent = StateGraph(State)
email_agent = email_agent.add_node(triage_router)
email_agent = email_agent.add_node("response_agent", response_agent)
email_agent = email_agent.add_edge(START, "triage_router")
email_agent = email_agent.compile(store=store)

## Setup Agent to update Long Term Memory in the background

Your email_agent is now setup to pull its instructions from long-term memory.

Now, you'll create an agent to update that memory.

### First check current behavior

In [16]:
email_input = {
    "author": "Alice Jones <alice.jones@bar.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "email_thread": """Hi John,

Urgent issue - your service is down. Is there a reason why""",
}

In [17]:
response = email_agent.invoke(
    {"email_input": email_input},
    config=config
)

📧 Classification: RESPOND - This email requires a response


In [18]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Respond to the email {'author': 'Alice Jones <alice.jones@bar.com>', 'to': 'John Doe <john.doe@company.com>', 'subject': 'Quick question about API documentation', 'email_thread': 'Hi John,\n\nUrgent issue - your service is down. Is there a reason why'}
================================== Ai Message ==================================

Okay, I'll respond to Alice Jones about the service being down. I will ask for more details to understand the issue better.
Tool Calls:
  write_email (ba2a5ca2-f9bb-490d-bf46-61d4ded720c8)
 Call ID: ba2a5ca2-f9bb-490d-bf46-61d4ded720c8
  Args:
    to: alice.jones@bar.com
    content: Hi Alice,

Thanks for the heads up. I'm sorry to hear about the service disruption. Could you please provide more details about the issue you are experiencing? Knowing the specific error messages or the time you encountered the problem will help us investigate this promptly.

Thanks,
John
    subj

and look at current values of long term memory

In [19]:
store.get(("lance",), "agent_instructions").value['prompt']

"Use these tools when appropriate to help manage John's tasks efficiently."

In [20]:
store.get(("lance",), "triage_respond").value['prompt']

'Direct questions from team members, meeting requests, critical bug reports'

In [21]:
store.get(("lance",), "triage_ignore").value['prompt']

'Marketing newsletters, spam emails, mass company announcements'

In [22]:
store.get(("lance",), "triage_notify").value['prompt']

'Team member out sick, build system notifications, project status updates'

### Now, Use an LLM to update instructions

WARNING: below code for Prompt Optimization doesn't work with Gemini for now

It fails with error:
`ValueError: Received unsupported arguments {'method': 'json_schema'}`

In [24]:
conversations = [
    (
        response['messages'],
        "Always sign your emails `John Doe`"
    )
]

In [25]:
prompts = [
    {
        "name": "main_agent",
        "prompt": store.get(("lance",), "agent_instructions").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on how the agent should write emails or schedule events"
        
    },
    {
        "name": "triage-ignore", 
        "prompt": store.get(("lance",), "triage_ignore").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails should be ignored"

    },
    {
        "name": "triage-notify", 
        "prompt": store.get(("lance",), "triage_notify").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails the user should be notified of"

    },
    {
        "name": "triage-respond", 
        "prompt": store.get(("lance",), "triage_respond").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails should be responded to"

    },
]

In [28]:
from langmem import create_multi_prompt_optimizer
from models import get_foundation_model

optimizer_llm = get_foundation_model()
optimizer = create_multi_prompt_optimizer(
    optimizer_llm,
    kind="prompt_memory",
)

ValueError: Received unsupported arguments {'method': 'json_schema'}

In [ ]:
updated = optimizer.invoke(
    {"trajectories": conversations, "prompts": prompts}
)

In [ ]:
print(updated)

In [ ]:
#json dumps is a bit easier to read
import json
print(json.dumps(updated, indent=4))